In [24]:
# importing libraries
import pandas as pd
import boto3
import numpy as np

# defining the bucket
s3 = boto3.resource('s3')
bucket_name = 'webster-data445-bucket'
bucket = s3.Bucket(bucket_name)

# defining the csv file
file_key = 'framingham.csv'

bucket_object = bucket.Object(file_key)
file_object = bucket_object.get()
file_content_stream = file_object.get('Body')

# reading the csv file
heart = pd.read_csv(file_content_stream)
heart.head()

# removing missing values
heart = heart.dropna()

# splitting data into training and testing
from sklearn.model_selection import train_test_split # importing train_test_split

X = heart[['age', 'currentSmoker', 'totChol', 'BMI', 'heartRate']]
Y = heart['TenYearCHD']

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2)

# building the linear model
from sklearn.linear_model import LogisticRegression

logit_md = LogisticRegression().fit(X_train, Y_train)

# predicting on test
preds = logit_md.predict_proba(X_test)[:,1]
preds

# changing values to 0 and 1
preds = np.where(preds < 0.25, 0, 1)

# importing confusion_matrix and accuracy_score
from sklearn.metrics import confusion_matrix, accuracy_score

# creating the confusion matrix
confusion_matrix(Y_test, preds)

# computing accuracy score
accuracy_score(Y_test, preds)

0.7978142076502732